In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.precision', 2)

## Бездефицитная система с мгновенным пополнением и постоянным спросом

Cf. https://en.wikipedia.org/wiki/Economic_order_quantity

In [2]:
r, K = 51, 231
q0, h0, = 0, 121
Q_STEP = 100
q = [q0 + k*Q_STEP for k in range(11)]
h = [h0*(1 - k/10) for k in range(11)]

df = pd.DataFrame({'q': q, 'h': h}, columns=['q', 'h'])
df.transpose()

,0,1,2,3,4,5,6,7,8,9,10
q,0.0,100.0,200.0,300.0,400.0,500.0,600.0,700.0,800.0,900.0,1000.0
h,121.0,108.9,96.8,84.7,72.6,60.5,48.4,36.3,24.2,12.1,0.0


$$
    q_\star = \sqrt{
    \frac{2rK}{h}}, $$

$$
    T_\star = \sqrt{
    \frac{2K}{rh}}, $$

$$J_\star = \sqrt{2rhK}$$

In [3]:
df['q_s'] = (2*r*K/df.h).pow(.5)
df['T_s'] = (2*K/r/df.h).pow(.5)
df['J_s'] = (2*r*K*df.h).pow(.5)
df.transpose()

,0,1,2,3,4,5,6,7,8,9,10
q,0.00,100.00,200.00,300.00,400.00,500.00,600.00,700.00,800.00,900.00,1000.00
h,121.00,108.90,96.80,84.70,72.60,60.50,48.40,36.30,24.20,12.10,0.00
q_s,13.95,14.71,15.60,16.68,18.02,19.73,22.06,25.48,31.20,44.13,inf
T_s,0.27,0.29,0.31,0.33,0.35,0.39,0.43,0.50,0.61,0.87,inf
J_s,1688.49,1601.84,1510.23,1412.69,1307.90,1193.94,1067.90,924.82,755.12,533.95,0.00


In [4]:
under_q = df.q_s < df.q
over_q = df.q_s > df.q + Q_STEP # inclusive?
well_q = ~(under_q | over_q)
df.loc[under_q, 'q_opt'] = df.loc[under_q, 'q']
df.loc[over_q, 'q_opt'] = Q_STEP + df.loc[over_q, 'q']
df.loc[well_q, 'q_opt'] = df.loc[well_q, 'q_s']
df['J(q_opt)'] = df.q_opt*df.h/2 + K*r/df.q_opt
df.transpose()

,0,1,2,3,4,5,6,7,8,9,10
q,0.00,100.00,200.00,300.00,400.00,500.00,600.00,700.00,800.00,900.00,1000.00
h,121.00,108.90,96.80,84.70,72.60,60.50,48.40,36.30,24.20,12.10,0.00
q_s,13.95,14.71,15.60,16.68,18.02,19.73,22.06,25.48,31.20,44.13,inf
T_s,0.27,0.29,0.31,0.33,0.35,0.39,0.43,0.50,0.61,0.87,inf
J_s,1688.49,1601.84,1510.23,1412.69,1307.90,1193.94,1067.90,924.82,755.12,533.95,0.00
q_opt,13.95,100.00,200.00,300.00,400.00,500.00,600.00,700.00,800.00,900.00,1100.00
J(q_opt),1688.49,5562.81,9738.91,12744.27,14549.45,15148.56,14539.64,12721.83,9694.73,5458.09,10.71
